# Parcial 3 - Experimentos

## Configurando ambiente de trabalho

In [ ]:
!git clone https://github.com/FerDelbo/beir.git

fatal: destination path 'beir' already exists and is not an empty directory.


In [ ]:
%cd /content/beir
!pwd

/content/beir
/content/beir


In [ ]:
!pip install pyserini faiss-cpu transformers --quiet

In [ ]:
# 1. Instala o Java 21
!sudo apt-get update -y --quiet
!sudo apt-get install openjdk-21-jdk -y --quiet

# 2. Define o Java 21 como a versão padrão
!sudo update-alternatives --set java /usr/lib/jvm/java-21-openjdk-amd64/bin/java --quiet

# 3. Verifica a versão (deve mostrar "21.x.x")
!java -version

Hit:1 https://cli.github.com/packages stable InRelease
Hit:2 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:5 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:6 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:7 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:8 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:11 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Reading package lists...
W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
Reading package lis

In [ ]:
# # 1. Clona o repositório oficial do trec_eval
!git clone https://github.com/usnistgov/trec_eval.git

# # 2. Entra no diretório e compila o programa (usando 'make')
%cd trec_eval
!make

# # 3. Copia o programa compilado para um local onde o sistema o encontre
!sudo cp trec_eval /usr/local/bin/

fatal: destination path 'trec_eval' already exists and is not an empty directory.
/content/beir/trec_eval
make: 'trec_eval' is up to date.


In [ ]:
%cd ..
!pwd

/content/beir
/content/beir


In [ ]:
!pip install -e . --quiet

  Installing build dependencies ... done
  Checking if build backend supports build_editable ... done
  Getting requirements to build editable ... done
  Preparing editable metadata (pyproject.toml) ... done
  Building editable for beir (pyproject.toml) ... done


## Verificando os exemplos

Exemplo com uma busca densa

In [ ]:
!python /content/beir/examples/retrieval/evaluation/dense/evaluate_ance.py

2025-10-27 20:37:18.272576: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1761597438.287465    2569 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1761597438.291636    2569 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1761597438.303060    2569 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1761597438.303111    2569 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1761597438.303115    2569 computation_placer.cc:177] computation placer alr

## Utilizando o pyserini

In [ ]:
from pyserini.search import get_qrels_file

# Isso vai encontrar o arquivo de gabarito (qrels) no cache do Pyserini
qrels_path = get_qrels_file('beir-v1.0.0-trec-covid-test')

print(f"Caminho do gabarito encontrado: {qrels_path}")

# Salva o caminho em uma variável de ambiente para usarmos no terminal
%env QRELS_PATH={qrels_path}

Caminho do gabarito encontrado: /root/.cache/pyserini/topics-and-qrels/qrels.beir-v1.0.0-trec-covid.test.txt
env: QRELS_PATH=/root/.cache/pyserini/topics-and-qrels/qrels.beir-v1.0.0-trec-covid.test.txt


Léxica - BM25 flat

In [ ]:
!python -m pyserini.search.lucene \
  --threads 16 --batch-size 128 \
  --index beir-v1.0.0-trec-covid.flat \
  --topics beir-v1.0.0-trec-covid-test \
  --output run.beir.bm25-flat.trec-covid.txt \
  --output-format trec \
  --hits 1000 --bm25 --remove-query

2025-10-27 21:46:41.391305: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1761601601.425166   19235 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1761601601.434301   19235 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1761601601.467725   19235 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1761601601.467807   19235 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1761601601.467816   19235 computation_placer.cc:177] computation placer alr

In [ ]:
!trec_eval -c -m ndcg_cut.10 $QRELS_PATH run.beir.bm25-flat.trec-covid.txt

ndcg_cut_10           	all	0.5947


In [ ]:
!trec_eval -c -m recall.100 $QRELS_PATH run.beir.bm25-flat.trec-covid.txt

recall_100            	all	0.1091


Densa - Contriever

In [ ]:
!python -m pyserini.search.faiss \
  --threads 16 --batch-size 512 \
  --encoder-class contriever --encoder facebook/contriever-msmarco \
  --index beir-v1.0.0-trec-covid.contriever-msmarco \
  --topics beir-v1.0.0-trec-covid-test \
  --output run.beir.contriever-msmarco.trec-covid.txt \
  --hits 1000 --remove-query

2025-10-27 21:47:16.214249: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1761601636.239088   19435 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1761601636.246071   19435 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1761601636.263573   19435 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1761601636.263633   19435 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1761601636.263640   19435 computation_placer.cc:177] computation placer alr

In [ ]:
!trec_eval -c -m ndcg_cut.10 $QRELS_PATH run.beir.contriever-msmarco.trec-covid.txt

ndcg_cut_10           	all	0.5964


In [ ]:
!trec_eval -c -m recall.100 $QRELS_PATH run.beir.contriever-msmarco.trec-covid.txt

recall_100            	all	0.0907


Esparsa - Slade


In [ ]:
!python -m pyserini.search.lucene \
  --threads 16 --batch-size 128 \
  --index beir-v1.0.0-trec-covid.splade-pp-ed \
  --topics beir-v1.0.0-trec-covid.test.splade-pp-ed \
  --output run.beir.splade-pp-ed.trec-covid.txt \
  --output-format trec \
  --hits 1000 --impact --remove-query


2025-10-27 21:48:06.222632: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1761601686.238558   19692 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1761601686.243804   19692 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1761601686.256051   19692 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1761601686.256101   19692 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1761601686.256106   19692 computation_placer.cc:177] computation placer alr

In [ ]:
!trec_eval -c -m ndcg_cut.10 $QRELS_PATH run.beir.splade-pp-ed.trec-covid.txt

ndcg_cut_10           	all	0.7274


In [ ]:
!trec_eval -c -m recall.100 $QRELS_PATH run.beir.splade-pp-ed.trec-covid.txt

recall_100            	all	0.1282


BGE

In [ ]:
!python -m pyserini.search.faiss \
  --threads 16 --batch-size 512 \
  --encoder-class auto --encoder BAAI/bge-base-en-v1.5 --l2-norm \
  --query-prefix "Represent this sentence for searching relevant passages:" \
  --index beir-v1.0.0-trec-covid.bge-base-en-v1.5 \
  --topics beir-v1.0.0-trec-covid-test \
  --output run.beir.bge-base-en-v1.5.faiss.trec-covid.txt \
  --hits 1000 --remove-query


2025-10-27 21:48:34.266135: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1761601714.281678   19856 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1761601714.286379   19856 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1761601714.300262   19856 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1761601714.300310   19856 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1761601714.300314   19856 computation_placer.cc:177] computation placer alr

In [ ]:
!trec_eval -c -m ndcg_cut.10 $QRELS_PATH run.beir.bge-base-en-v1.5.faiss.trec-covid.txt

ndcg_cut_10           	all	0.7815


In [ ]:
!trec_eval -c -m recall.100 $QRELS_PATH run.beir.bge-base-en-v1.5.faiss.trec-covid.txt

recall_100            	all	0.1406


Cohere

In [ ]:
!python -m pyserini.search.faiss \
  --threads 16 --batch-size 512 \
  --index beir-v1.0.0-trec-covid.cohere-embed-english-v3.0  \
  --topics beir-v1.0.0-trec-covid-test \
  --encoded-queries cohere-embed-english-v3.0-beir-v1.0.0-trec-covid-test \
  --output run.beir.cohere-embed-english-v3.0.trec-covid.txt \
  --hits 1000 --remove-query


2025-10-27 21:49:21.214675: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1761601761.230375   20095 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1761601761.234901   20095 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1761601761.246988   20095 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1761601761.247039   20095 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1761601761.247044   20095 computation_placer.cc:177] computation placer alr

In [ ]:
!trec_eval -c -m ndcg_cut.10 $QRELS_PATH run.beir.cohere-embed-english-v3.0.trec-covid.txt

ndcg_cut_10           	all	0.8178


In [ ]:
!trec_eval -c -m recall.100 $QRELS_PATH run.beir.cohere-embed-english-v3.0.trec-covid.txt

recall_100            	all	0.1594
